<a href="https://colab.research.google.com/github/2015samtaylor/Green_Dot/blob/main/CA_Enroll_CDE_CensusEnroll_Cnt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Census Day Enrollment File & CDE Dataquest Subgroups

import pandas as pd
pd.options.mode.chained_assignment = None
from urllib.request import urlretrieve as retrieve
import time
start_time = time.time()
import numpy as np


# grabs file from URL below, and transforms file to dataframe

url ='https://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=School&cYear=2122&cCat=ShortTermEnrl&cPage=filesenrps.asp'
retrieve(url, 'test.txt')
df = pd.read_table('test.txt')

schools = ['Alain Leroy Locke College Preparatory Academy', 'Animo City of Champions Charter High', 'Animo Compton Charter',
          'Animo Ellen Ochoa Charter Middle', 'Animo Florence-Firestone Charter Middle', 'Animo Inglewood Charter High',
          'Animo Jackie Robinson High', 'Animo James B. Taylor Charter Middle', 'Animo Jefferson Charter Middle', 
          'Animo Leadership High', 'Animo Legacy Charter Middle', 'Animo Mae Jemison Charter Middle', 'Animo Pat Brown',
          'Animo Ralph Bunche Charter High', 'Animo South Los Angeles Charter', 'Animo Venice Charter High', 'Animo Watts College Preparatory Academy',
          'Animo Westside Charter Middle', 'Oscar De La Hoya Animo Charter High']

school_codes = ['LCK', 'CHA', 'CMP', 'AEO', 'FLO', 'ING', 'ROB', 'JAM', 'JMS', 'LEA', 'LGC', 'MAE', 'BRW', 'BUN', 'SLA',
               'VEN', 'WAT', 'DLH']

race_dic = {0 :'Not reported', 1 : 'American Indian or Alaska Native', 2 : 'Asian', 3 : 'Pacific Islander', 4 : 'Filipino',
        5 : 'Hispanic or Latino', 6 : 'African American', 7 : 'White', 9 : 'Two or More Races'}


school_dic = dict(zip(schools, school_codes))
schools_filt = list(school_dic.keys())


# apply dictionary to get all green dot schools

GD = df.loc[(df['SCHOOL'].isin(schools_filt))]

# add in school id column 
GD['CDS_CODE'] = GD['CDS_CODE'].astype(str)
GD['SchoolID'] = GD['CDS_CODE'].str[-7:]

# replace full school Names with school abbreviation
GD.replace({'SCHOOL': school_dic, 'ETHNIC' : race_dic}, inplace = True)

groups = GD.groupby('SCHOOL')

def school_info(i):
       
    school_name = groups.get_group(i)


    school_total_enroll = school_name['ENR_TOTAL'].sum()
    total_females = school_name['ENR_TOTAL'].loc[(school_name['GENDER'] == 'F')].sum()
    total_males = school_name['ENR_TOTAL'].loc[(school_name['GENDER'] == 'M')].sum()
    other = school_total_enroll - (total_females + total_males)


    males = {'School': i, 'GroupType': 'Gender', 'Subgroup': 'Male', 'DataValue' : total_males  }
    females = {'School': i, 'GroupType': 'Gender', 'Subgroup': 'Female', 'DataValue' : total_females  }
    other = {'School': i, 'GroupType': 'Gender', 'Subgroup': 'X', 'DataValue' : other  }
    total = {'School': i, 'GroupType' : 'All', 'Subgroup': 'All Students', 'DataValue' : school_total_enroll}

    six = school_name['GR_6'].sum()
    seven = school_name['GR_7'].sum()
    eight = school_name['GR_8'].sum()
    nine = school_name['GR_9'].sum()
    ten = school_name['GR_10'].sum()
    eleven = school_name['GR_11'].sum()
    twelve = school_name['GR_12'].sum()
    
    sixth = {'School': i, 'GroupType': 'Grade Level', 'Subgroup': '6', 'DataValue': six}
    seventh = {'School': i, 'GroupType': 'Grade Level', 'Subgroup': '7', 'DataValue': seven}
    eigth = {'School': i, 'GroupType': 'Grade Level', 'Subgroup': '8', 'DataValue': eight}
    ninth = {'School': i, 'GroupType': 'Grade Level', 'Subgroup': '9', 'DataValue': nine}
    tenth = {'School': i, 'GroupType': 'Grade Level', 'Subgroup': '10', 'DataValue' : ten}
    eleventh = {'School': i, 'GroupType': 'Grade Level', 'Subgroup': '11', 'DataValue' : eleven}
    twelfth = {'School': i, 'GroupType': 'Grade Level', 'Subgroup': '12', 'DataValue' : twelve}
    
    
    AA = school_name['ENR_TOTAL'].loc[(school_name['ETHNIC'] == 'African American')].sum()
    Hispanic_or_Latino = school_name['ENR_TOTAL'].loc[(school_name['ETHNIC'] == 'Hispanic or Latino')].sum()
    Two_or_More_Races = school_name['ENR_TOTAL'].loc[(school_name['ETHNIC'] == 'Two or More Races')].sum()
    White = school_name['ENR_TOTAL'].loc[(school_name['ETHNIC'] == 'White')].sum()
    Asian = school_name['ENR_TOTAL'].loc[(school_name['ETHNIC'] == 'Asian')].sum()
    Not_Reported = school_name['ENR_TOTAL'].loc[(school_name['ETHNIC'] == 'Not Reported')].sum()
    

    AA = {'School': i, 'GroupType': 'Race_Ethnicity', 'Subgroup': 'AA', 'DataValue' : AA}
    Hispanic_or_Latino = {'School': i, 'GroupType': 'Race_Ethnicity', 'Subgroup': 'Hispanic or Latino', 'DataValue' : Hispanic_or_Latino}
    Two_or_More_Races = {'School': i, 'GroupType': 'Race_Ethnicity', 'Subgroup': 'Two or More Races', 'DataValue' : Two_or_More_Races}
    White = {'School': i, 'GroupType': 'Race_Ethnicity', 'Subgroup': 'White', 'DataValue' : White}
    Asian = {'School': i, 'GroupType': 'Race_Ethnicity', 'Subgroup': 'Asian', 'DataValue' : Asian}
    Not_Reported = {'School': i, 'GroupType': 'Race_Ethnicity', 'Subgroup': 'Not Reported', 'DataValue' : Not_Reported}
        
    return(males, females, other, total, sixth, seventh, eigth, ninth, tenth, eleventh, twelfth, AA, Hispanic_or_Latino, Two_or_More_Races, White, Asian, Not_Reported)



# run for all school codes in a loop, and create concated df  
school_df = []

list_len = len(school_codes)
i = 0
while i < list_len:

    output = pd.DataFrame(school_info(school_codes[i]))
    i += 1
    school_df.append(output)

df = pd.concat(school_df).reset_index(drop = True)

# fixed bugs divides values from loop in half & drops duplicate subgroup of ALl Students which is pulled from cde subgroups

df.drop(df.loc[df['Subgroup'] == 'All Students'].index, inplace = True)
df['DataValue'] = (df['DataValue']/2).astype(int)


# -------------------------------------------get info from cde

syms = list(GD['SCHOOL'].unique())
cds = list(GD['CDS_CODE'].unique())

aggs = []


def get_info():
    i = 0
    list_len = len(syms)

    while i < list_len:
        url = ('https://dq.cde.ca.gov/dataquest/dqcensus/EnrCharterSub.aspx?cds={}&agglevel=School&year=2021-22&ro=y&ro=y&ro=y').format(cds[i])
        output = pd.read_html(url)[1]
        output.insert(0, 'School', syms[i])
        aggs.append(output)

        i += 1

get_info()    
    
cde = pd.concat(aggs).reset_index(drop = True)
cde.rename(columns = {'Enrollment' : 'DataValue'}, inplace = True)

grouptype_dic = {'Foster Youth' : 'Foster',
'Homeless Youth' : 'Homeless',
'Students with Disabilities' : 'SWD',
'Socioeconomically Disadvanted' : 'SED',
'English Learners' : 'EL',
'Socioeconomically Disadvantaged': 'SED',
'Migrant Education':'Migrant Education',
'All Students' : 'All'}

cde['GroupType'] = cde['Subgroup'].map(grouptype_dic)

GroupType = cde.pop('GroupType')
cde.insert(1, 'GroupType', GroupType)
# --------------------------------------------------- get ela info 


# GET ELA status

aggs = []


def get_info():
    i = 0
    list_len = len(syms)

    while i < list_len:
        url = 'https://dq.cde.ca.gov/dataquest/dqcensus/EnrELASLevels.aspx?cds={}&agglevel=School&year=2021-22&ro=y&ro=y&ro=y&ro=y&ro=y'.format(cds[i])
        output = pd.read_html(url)[1]
        output.insert(0, 'School', syms[i])
        
        
        lst = ['English Only (EO)','Initial Fluent English Proficient (IFEP)', 'English Learner (EL)',
        'Reclassified Fluent English Proficient (RFEP)', 'To Be Determined (TBD)']

        for column in lst:
            output[column] = output[column].replace({'\%': ''}, regex = True)
    
        for column in lst:
            output[column] = pd.to_numeric(output[column])/100
            
        
        iterate = output.iloc[:1, 3:]
        preserve = output['Total'][0]
        new_vals = np.round(iterate.values * preserve).astype(int)
        output.iloc[:1, 3:] = new_vals
        
        aggs.append(output)

        i += 1

get_info()    

el = pd.concat(aggs).reset_index(drop = True)

col_dic = {'English Only (EO)' : 'EO', 'Initial Fluent English Proficient (IFEP)' : 'IFEP',
                    'English Learner (EL)' : 'EL', 'Reclassified Fluent English Proficient (RFEP)': 'RFEP', 
                    'To Be Determined (TBD)' : 'TBD'}

el.rename(columns = col_dic, inplace = True)

col_list = list(col_dic.values())

output = []

list_len = len(col_list)
i = 0

while i < list_len:
    melt = el[['School', col_list[i]]]
    out = pd.melt(melt, id_vars = ['School'])
    output.append(out)
    i += 1

el = pd.concat(output).reset_index(drop = True)   

col_dic = {'variable' : 'Subgroup', 'value' : 'DataValue'}
el.rename(columns = col_dic, inplace = True)
el.insert(1, 'GroupType', 'ELA Status')

df = pd.concat([df, cde, el], ignore_index = True)

# ---------------------------------------------------add in final columns. 

df.insert(3, 'SchoolYear', '2021-2022')
df.insert(0, 'MetricID', 'CA_Enroll_CDE_CensusEnroll_Cnt')

# map the SchoolID codes to the dataframe

School = ['CHA', 'ING', 'LEA', 'DLH', 'SLA', 'VEN', 'BRW', 'BUN', 'ROB',
       'WAT', 'LCK', 'JMS', 'WMS', 'AEO', 'JAM', 'LGC', 'MAE', 'FLO',
       'CMP']

SchoolID = [ 136119, 1996586, 1996313,  101675,  102434,  106831,  106849,
        111575,  111583,  111625,  118588,  122481,  122499,  123992,
        124008,  124016,  129270,  134023,  137984]

school_dict = dict(zip(School, SchoolID))
df['SchoolID'] = df['School'].map(school_dict)

SchoolID = df.pop('SchoolID')
df.insert(2, 'SchoolID', SchoolID)

df.drop(df.loc[df['GroupType'] == 'Migrant Education'].index, inplace = True)
df['DataValue'] = df['DataValue'].astype(int)

print('--- %s seconds ----' % (time.time() - start_time))

df

--- 45.215131521224976 seconds ----


,MetricID,School,SchoolID,GroupType,Subgroup,SchoolYear,DataValue
0,CA_Enroll_CDE_CensusEnroll_Cnt,LCK,118588,Gender,Male,2021-2022,733
1,CA_Enroll_CDE_CensusEnroll_Cnt,LCK,118588,Gender,Female,2021-2022,554
2,CA_Enroll_CDE_CensusEnroll_Cnt,LCK,118588,Gender,X,2021-2022,0
3,CA_Enroll_CDE_CensusEnroll_Cnt,LCK,118588,Grade Level,6,2021-2022,0
4,CA_Enroll_CDE_CensusEnroll_Cnt,LCK,118588,Grade Level,7,2021-2022,0
...,...,...,...,...,...,...,...
499,CA_Enroll_CDE_CensusEnroll_Cnt,BUN,111575,ELA Status,TBD,2021-2022,0
500,CA_Enroll_CDE_CensusEnroll_Cnt,LGC,124016,ELA Status,TBD,2021-2022,3
501,CA_Enroll_CDE_CensusEnroll_Cnt,MAE,129270,ELA Status,TBD,2021-2022,0
502,CA_Enroll_CDE_CensusEnroll_Cnt,FLO,134023,ELA Status,TBD,2021-2022,0


In [2]:
#@title Download as a CSV file

from google.colab import files
df.to_csv('CA_Enroll_CDE_CensusEnroll_Cnt.csv')
files.download('CA_Enroll_CDE_CensusEnroll_Cnt.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>